In [1]:
import requests
import os # Interoperable file paths
import pathlib  # Work with file and directory paths
import re # Extract metadata from file names
from math import floor, ceil

from glob import glob # Find files by pattern
from io import BytesIO # Stream binary (zip) files

import geopandas as gpd # Work with geospatial data
import rasterio as rio  # Read, write, and manipulate raster datasets
import rasterio.mask # Selectively extract portions of a raster dataset 
import rioxarray as rxr
import subprocess

In [2]:
%store -r

In [3]:
# Define and create the project data directories
data_dir = os.path.join(
    pathlib.Path.home(),
    'GitHub',
    'habitat_project',
)
os.makedirs(data_dir, exist_ok=True)

In [5]:
# Access Polaris Data for Cherokee County OK
ok_water_url_template =(
    "http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0"
    "/ksat"
    "/mean"
    "/0-5/"
    "/lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif"
)

ok_water_url = ok_water_url_template.format(
    min_lat=35, max_lat=37, min_lon=-95, max_lon=-94
)
ok_water_url

'http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/ksat/mean/0-5//lat3537_lon-95-94.tif'

In [6]:
bounds_min_lon, bounds_min_lat, bounds_max_lon, bounds_max_lat = (
    cherokee_county.total_bounds)

ok_water_url_list = []
for min_lon in range(floor(bounds_min_lon), ceil(bounds_max_lon)):
    for min_lat in range(floor(bounds_min_lat), ceil(bounds_max_lat)):
        ok_water_url = ok_water_url_template.format(
            min_lat=min_lat, max_lat=min_lat+1,
            min_lon=min_lon, max_lon=min_lon+1)
        ok_water_url_list.append(ok_water_url)

ok_water_url_list

['http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/ksat/mean/0-5//lat3536_lon-96-95.tif',
 'http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/ksat/mean/0-5//lat3637_lon-96-95.tif',
 'http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/ksat/mean/0-5//lat3536_lon-95-94.tif',
 'http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/ksat/mean/0-5//lat3637_lon-95-94.tif']

In [7]:
ok_water_da = rxr.open_rasterio(
    ok_water_url,
    mask_and_scale=True
).squeeze()
ok_water_da

RasterioIOError: HTTP response code: 404